In [21]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json

In [110]:
headers = {
            "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/85.0.4183.102 Safari/537.36', 'Accept-Language': 'en-US'#'zh-CN'
        }

In [43]:
def get_url_list(n):
    linklist=[]
    IDlist = []
    for pagenum in range(1,n):
        r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%pagenum,headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"),class_="search_result_row ds_collapse_flag")
        for i in soups:
            i = i.attrs
            i = i['href']
            link = re.search('https://store.steampowered.com/app/(\d*?)/',i).group()
            ID = re.search('https://store.steampowered.com/app/(\d*?)/(.*?)/', i).group(1)
            linklist.append(link)
            IDlist.append(ID)
        print(str(pagenum)+' pages completed, now '+str(len(linklist)))
    #print(linklist)
    return linklist,IDlist

In [45]:
def getdf(n):#turn the result to df
    linklist,IDlist = get_url_list(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)),
               columns =['Link', 'ID'])
    #print(df)
    return df

In [97]:
def gamename(soup):   #name of game
    try:
        a = soup.find(class_="apphub_AppName")
        k = str(a.string)
    except:
        a = soup.find(class_="apphub_AppName")
        k = str(a.text)
    return k

In [ ]:
"Racing\nAction\nMultiplayer\nCharacter Customization\nSingleplayer\nCombat Racing\nOpen World\nDriving\nAnime\nArcade\nCasual\nController\nOnline Co-Op\nSimulation\nMusic\nAdventure\nAtmospheric\nSplit Screen\nSports\nImmersive".split("\n")

In [119]:
def taglist(soup):#tag list
    list1=[]
    a = soup.find_all(class_="app_tag")
    for i in a:
        k = str(i.string).replace('	', '').replace('\n', '').replace('\r', '')
        if k == '+':
            pass
        else:
            list1.append(k)
    list1 = str('\n'.join(list1)).split("\n")
    return list1

In [115]:
def gameprice(soup):#price
    try:
        a = soup.findAll(class_="discount_original_price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    except:
        a = soup.findAll(class_="game_purchase_price price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    if(re.search('free', k,re.IGNORECASE)):
        return 0
    else:
        return float(k[1:])

In [121]:
def gamerate(soup):#get rating
    a = soup.find(class_="user_reviews_summary_row")
    k = str((a.attrs)['data-tooltip-html'])
    return float(k.split("%")[0])/100

In [117]:
def open_cache(CACHE_FILENAME):
    try:
        cache_file = open(CACHE_FILENAME, 'r')
        cache_contents = cache_file.read()
        cache_dict = json.loads(cache_contents)
        cache_file.close()
    except:
        cache_dict = []
    return cache_dict
def write_cache(game_json, CACHE_FILENAME):
    dumped_json_cache = json.dumps(game_json, indent=2)
    fw = open(CACHE_FILENAME,"w")
    fw.write(dumped_json_cache)
    fw.close()

In [ ]:
a = requests.get("https://store.steampowered.com/app/1938090/", headers=headers,timeout=10)
print(a.text)


In [143]:

game_all = {}
tag_all = {}
price_all = {
    "free":[],
    "0-30":[],
    "30-100":[],
    "100-200":[],
    "more expensive than 200":[]
}
rate_all = {
    "0\%-20\%":[],
    "20\%-40\%":[],
    "40\%-60\%":[],
    "60\%-80\%":[],
    "80\%-100\%":[]
}
cache = open_cache("cache.json")
if len(cache) == 0:
    df = getdf(130)
    for url in df['Link']:
        if(len(game_all) >= 2000):
            break
        if(len(game_all)%100 == 0):
            print(len(game_all))
        a = requests.get(url, headers=headers,timeout=10)
        soup = BeautifulSoup(a.text, 'lxml')
        try:
            n = gamename(soup)
            tags = taglist(soup)
            p = gameprice(soup)
            rate = gamerate(soup)
            game_all[n] = {
                "taglist" : tags,
                "price":p,
                "rate":rate
            }
            for tag in tags:
                if tag in tag_all:
                    tag_all[tag].append(n)
                else:
                    tag_all[tag] = [n]
            if p == 0:
                price_all["free"].append(n)
            elif p < 30:
                price_all["0-30"].append(n)
            elif p < 100:
                price_all["30-100"].append(n)
            elif p < 200:
                price_all["100-200"].append(n)
            else:
                price_all["more expensive than 200"].append(n)
            match int(rate*10):
                case 0:
                    rate_all["0\%-20\%"].append(n)
                case 1:
                    rate_all["0\%-20\%"].append(n)
                case 2:
                    rate_all["20\%-40\%"].append(n)
                case 3:
                    rate_all["20\%-40\%"].append(n)
                case 4:
                    rate_all["40\%-60\%"].append(n)
                case 5:
                    rate_all["40\%-60\%"].append(n)
                case 6:
                    rate_all["60\%-80\%"].append(n)
                case 7:
                    rate_all["60\%-80\%"].append(n)
                case 8:
                    rate_all["80\%-100\%"].append(n)
                case 9:
                    rate_all["80\%-100\%"].append(n)
                case 10:
                    rate_all["80\%-100\%"].append(n)
        except:
            continue
    
    write_cache(game_all, "cache.json")
else:
    for n in cache:
        tags = cache[n]["taglist"]
        for tag in tags:
            if tag in tag_all:
                tag_all[tag].append(n)
            else:
                tag_all[tag] = [n]
        p = cache[n]["price"]
        if p == 0:
            price_all["free"].append(n)
        elif p < 30:
            price_all["0-30"].append(n)
        elif p < 100:
            price_all["30-100"].append(n)
        elif p < 200:
            price_all["100-200"].append(n)
        else:
            price_all["more expensive than 200"].append(n)
        rate = cache[n]["rate"]
        match int(rate*10):
            case 0:
                rate_all["0\%-20\%"].append(n)
            case 1:
                rate_all["0\%-20\%"].append(n)
            case 2:
                rate_all["20\%-40\%"].append(n)
            case 3:
                rate_all["20\%-40\%"].append(n)
            case 4:
                rate_all["40\%-60\%"].append(n)
            case 5:
                rate_all["40\%-60\%"].append(n)
            case 6:
                rate_all["60\%-80\%"].append(n)
            case 7:
                rate_all["60\%-80\%"].append(n)
            case 8:
                rate_all["80\%-100\%"].append(n)
            case 9:
                rate_all["80\%-100\%"].append(n)
            case 10:
                rate_all["80\%-100\%"].append(n)



1 pages completed, now 24
2 pages completed, now 48
3 pages completed, now 72
4 pages completed, now 96
5 pages completed, now 120
6 pages completed, now 144
7 pages completed, now 169
8 pages completed, now 194
9 pages completed, now 217
10 pages completed, now 242
11 pages completed, now 265
12 pages completed, now 289
13 pages completed, now 313
14 pages completed, now 335
15 pages completed, now 360
16 pages completed, now 384
17 pages completed, now 409
18 pages completed, now 434
19 pages completed, now 455
20 pages completed, now 478
21 pages completed, now 502
22 pages completed, now 525
23 pages completed, now 549
24 pages completed, now 572
25 pages completed, now 596
26 pages completed, now 619
27 pages completed, now 644
28 pages completed, now 667
29 pages completed, now 691
30 pages completed, now 714
31 pages completed, now 739
32 pages completed, now 762
33 pages completed, now 786
34 pages completed, now 810
35 pages completed, now 833
36 pages completed, now 856
37 pa

In [161]:
while(True):
    print("Welcome to game searching")
    game = []
    while(True):
        try:
            p = float(input("Please give a price of the game that you intend, (0 indicate free game): "))
            break
        except:
            print("Please type a valid number")
    if p == 0:
        game = price_all["free"]
    elif p < 30:
        game = price_all["0-30"]
    elif p < 100:
        game = price_all["30-100"]
    elif p < 200:
        game = price_all["100-200"]
    else:
        game = price_all["more expensive than 200"]
    tags = []
    tag = input("Please give a tag that fits the game you want, i.e. FPS: ")
    tags.append(tag)
    while(True):
        tag = input("Please give a tag that fits the game you want, if you are finish giving tag, type \"exit\": ")
        if(tag == "exit"):
            break
        else:
            tags.append(tag)
    for tag in tags:
        game = list(set(tag_all[tag]) & set(game))
    print("The game we have for you is ",game)
    next_turn = input("Do you want another search?[Y\\N]: ")
    if(next_turn == "N"):
        break

    
    
    




Welcome to game searching
The game we have for you is  ['STAR WARS™ Battlefront™ II', 'Red Matter 2', 'Scorn', 'Generation Zero®', 'Destiny 2', 'Deep Rock Galactic', 'S.T.A.L.K.E.R.: Clear Sky', 'Space Hulk: Deathwing Enhanced Edition', 'Warhammer 40,000: Darktide', 'Prey', 'Battlezone: Combat Commander', "No Man's Sky", 'Genesis Alpha One Deluxe Edition', 'Osiris: New Dawn', 'Metro Exodus', 'Halo Infinite', 'STAR WARS™ Battlefront', 'Midnight Ghost Hunt', 'Apex Legends™', 'S.T.A.L.K.E.R.: Call of Pripyat']


In [160]:
#print(tag_all.keys())
#list(set(tag_all["FPS"]) & set(tag_all["Sci-fi"]))
try:
    price = float(input("hehe: "))
except:
    print("hehe")
price = price+1